In [2]:
!pip install torch torchvision
!pip install torch-geometric
!pip install osmnx

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 

In [3]:
import osmnx as ox
from shapely.geometry import Polygon, Point
import pandas as pd
import numpy as np

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load the map.

In [57]:
G = ox.load_graphml('/content/drive/MyDrive/RAC Project/map/perth-drive-con-unprojected.graphml')

nodes, edges = ox.graph_to_gdfs(G=G,nodes=True,edges=True)

Load the historic traffic data and match the road name by ID.

In [58]:
traffic_df = pd.read_csv('/content/drive/MyDrive/RAC Project/traffic_data/Total_Traffic_Volume.csv')

In [59]:
road_df = pd.read_csv('/content/drive/MyDrive/RAC Project/traffic_data/M-Links_Road_Network.csv', usecols=['LINK_DESCR', 'LINK_TO', 'LINK_ID'])
road_df['M_Link_ID'] = road_df['LINK_ID']

In [60]:
volume_df = traffic_df.merge(road_df[['M_Link_ID', 'LINK_DESCR', 'LINK_TO']], how='left', on='M_Link_ID')
volume_df = volume_df[['LINK_DESCR', 'LINK_TO', 'Volumes']]

In [61]:
def explode_link_to(df):
  df['LINK_TO'] = df['LINK_TO'].apply(lambda x: x.split('&') if isinstance(x, str) else x)
  # Explode DataFrame with handling for non-strings
  return df.explode('LINK_TO').reset_index(drop=True)

volume_df = explode_link_to(volume_df.copy())

In [62]:
def process_dataframe(df, f1, f2):
  df[f1] = df[f1].apply(lambda x: ' '.join(x.split()[:2]) if isinstance(x, str) else x)
  df[f2] = df[f2].apply(lambda x: ' '.join(x.split()[:2]) if isinstance(x, str) else x)
  return df.copy()

volume_df = process_dataframe(volume_df, 'LINK_DESCR', 'LINK_TO')

Load the crash information within the study area.

In [63]:
crash_df = pd.read_csv('/content/drive/MyDrive/RAC Project/crash_data/Crash_Information.csv',
                       usecols=['X', 'Y', 'INTERSECTION_DESC', 'SEVERITY'])
crash_df = crash_df.dropna(subset=['INTERSECTION_DESC'])

lon_values = [G.nodes[node]['x'] for node in G.nodes()]
lat_values = [G.nodes[node]['y'] for node in G.nodes()]

min_lon = min(lon_values)
max_lon = max(lon_values)
min_lat = min(lat_values)
max_lat = max(lat_values)

filtered_df = crash_df[(crash_df['X'] >= min_lon) &
                 (crash_df['X'] <= max_lon) &
                 (crash_df['Y'] >= min_lat) &
                 (crash_df['Y'] <= max_lat)]

Calculate the KSI metric.

In [64]:
def calculate_ksi(group):
    ksi_crash = group[(group['SEVERITY'] == 'Fatal') | (group['SEVERITY'] == 'Hospital')].shape[0]
    medical_crash = group[group['SEVERITY'] == 'Medical'].shape[0]
    casualty_crash = ksi_crash + medical_crash
    if casualty_crash == 0:
        return 0
    ksi_metric = ksi_crash + ksi_crash / casualty_crash * medical_crash
    return ksi_metric

# Apply the function to each group
ksi_metrics = filtered_df.groupby('INTERSECTION_DESC').apply(calculate_ksi).reset_index(name='KSI_metric')

# Merge the KSI metric back to the original dataframe
filtered_df = filtered_df.merge(ksi_metrics, on='INTERSECTION_DESC', how='left')
filtered_df = filtered_df.drop(['SEVERITY'], axis=1)

Extract the traffic volume of each node.

In [65]:
# Split the 'INTERSECTION_DESC' column based on '&' and create new columns
filtered_df[['MAJOR_ROAD', 'MINOR_ROAD']] = filtered_df['INTERSECTION_DESC'].str.split('&', n=1, expand=True)

# Use explode to expand the DataFrame based on 'INTERSECTION_DESC'
expanded_df = filtered_df.assign(MINOR_ROAD=filtered_df['MINOR_ROAD'].str.split('&')).explode('MINOR_ROAD')

# Reset index to maintain consecutive row numbers
expanded_df.reset_index(drop=True, inplace=True)

In [66]:
expanded_df = process_dataframe(expanded_df, 'MAJOR_ROAD', 'MINOR_ROAD')

In [67]:
expanded_df = expanded_df.drop(['INTERSECTION_DESC'], axis=1)

In [68]:
merged_df = expanded_df.merge(volume_df, left_on=['MAJOR_ROAD', 'MINOR_ROAD'], right_on=['LINK_DESCR', 'LINK_TO'], how='left')
merged_df = merged_df.drop(['LINK_DESCR', 'LINK_TO'], axis=1)

In [69]:
merged_df['Volumes'].fillna(0, inplace=True)

In [70]:
compressed_df = merged_df.groupby(['X', 'Y', 'KSI_metric', 'MAJOR_ROAD', 'MINOR_ROAD']).agg({'Volumes': 'mean'}).reset_index()

Calculate the crash rate.

In [71]:
compressed_df['CRASH_RATE'] = compressed_df['KSI_metric']*10**8/compressed_df['Volumes']/1.7

In [72]:
compressed_df['CRASH_RATE'].fillna(0, inplace=True)
compressed_df.loc[compressed_df['Volumes'] == 0, 'CRASH_RATE'] = 0

In [73]:
compressed_df = compressed_df.drop(['MAJOR_ROAD', 'MINOR_ROAD', 'Volumes'], axis=1)

In [74]:
compressed_df['x'] = compressed_df['X']
compressed_df['y'] = compressed_df['Y']
compressed_df = compressed_df.drop(['X', 'Y'], axis=1)

Identify the risk level (safe, low, medium or high).

In [75]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the 'CRASH_RATE' column
compressed_df[['KSI_metric', 'CRASH_RATE']] = scaler.fit_transform(compressed_df[['KSI_metric', 'CRASH_RATE']])

In [76]:
compressed_df = compressed_df[compressed_df['KSI_metric'] != 0]

In [77]:
ksi_threshold = compressed_df['KSI_metric'].quantile(0.3)
crash_rate_threshold = compressed_df['CRASH_RATE'].quantile(0.7)

# Classify based on thresholds
def classify(row):
    if row['KSI_metric'] > ksi_threshold and row['CRASH_RATE'] > crash_rate_threshold:
        return 3
    elif row['KSI_metric'] > ksi_threshold and row['CRASH_RATE'] <= crash_rate_threshold:
        return 2
    elif row['KSI_metric'] <= ksi_threshold and row['CRASH_RATE'] > crash_rate_threshold:
        return 1
    else:
        return 0

compressed_df['risk_lvl'] = compressed_df.apply(classify, axis=1)

label_counts = compressed_df['risk_lvl'].value_counts()
label_counts = label_counts.sort_index()
label_prob = label_counts / label_counts.sum()

In [78]:
compressed_df = compressed_df.drop(['KSI_metric', 'CRASH_RATE'], axis=1)

Prepare the training and testing data.

In [79]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Convert accident DataFrame to GeoDataFrame
accident_geometry = [Point(xy) for xy in zip(compressed_df['x'], compressed_df['y'])]
accident_gdf = gpd.GeoDataFrame(compressed_df, geometry=accident_geometry, crs='EPSG:4326')

In [80]:
import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import BallTree

# Convert the x, y coordinates from nodes to a numpy array
nodes_array = np.column_stack((nodes['x'], nodes['y']))

# Build a BallTree for nearest neighbor search
tree = BallTree(nodes_array, leaf_size=15)

matched_nodes = accident_gdf.copy()

# Define a function to find the nearest point and return its geometry
def find_nearest_geometry(row):
    point = np.array([[row['x'], row['y']]])
    dist, ind = tree.query(point, k=1)
    nearest_index = ind[0][0]
    return nodes.iloc[nearest_index]['geometry']

matched_nodes['geometry'] = matched_nodes.apply(find_nearest_geometry, axis=1)
accident_nodes = gpd.GeoDataFrame(matched_nodes, geometry=matched_nodes['geometry'], crs='EPSG:4326')

In [81]:
from sklearn.model_selection import train_test_split

X = accident_nodes.drop(columns=['risk_lvl'])
y = accident_nodes[['risk_lvl']]
train_nodes, test_nodes, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [82]:
train_nodes = gpd.sjoin(nodes, train_nodes, how="right", predicate="intersects")
train_nodes = train_nodes[['osmid_original', 'x_right', 'y_right', 'street_count', 'geometry']]
train_nodes = train_nodes.rename(columns={'x_right': 'x', 'y_right': 'y'})
test_nodes = gpd.sjoin(nodes, test_nodes, how="right", predicate="intersects")
test_nodes = test_nodes[['osmid_original', 'x_right', 'y_right', 'street_count', 'geometry']]
test_nodes = test_nodes.rename(columns={'x_right': 'x', 'y_right': 'y'})

In [83]:
train_edges = gpd.sjoin(edges, train_nodes, how="inner", predicate="intersects")
train_edges = train_edges[['osmid', 'highway', 'oneway', 'length', 'speed_kph', 'geometry']]
test_edges = gpd.sjoin(edges, test_nodes, how="inner", predicate="intersects")
test_edges = test_edges[['osmid', 'highway', 'oneway', 'length', 'speed_kph', 'geometry']]

In [84]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
import networkx as nx
import geopandas as gpd

G_train = nx.Graph(crs='EPSG:4326')
for idx, row in train_nodes.iterrows():
    G_train.add_node(idx, x=row['x'], y=row['y'])
for idx, row in train_edges.iterrows():
    G_train.add_edge(row.name[0], row.name[1], oneway=row['oneway'])

G_train.add_edges_from(nx.selfloop_edges(G_train))

x = torch.tensor(train_nodes[['x', 'y', 'street_count']].values, dtype=torch.float)
# Ensure that node indices in the edge index are within the range of the number of nodes
train_edges['oneway'] = train_edges['oneway'].astype(float)
edge_index = torch.tensor(np.array(list(G_train.edges())).T, dtype=torch.long)
edge_index = edge_index.remainder(len(train_nodes))  # Ensure node indices are within bounds
edge_attr = torch.tensor(train_edges[['oneway', 'length', 'speed_kph']].values, dtype=torch.float)

The architecture of the GCN model.

In [85]:
# Define GCN model
class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(3, 128)
        self.conv2 = GCNConv(128, 64)
        self.conv3 = GCNConv(64, 32)
        self.conv4 = GCNConv(32, 4)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, 0.1)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, 0.1)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, 0.1)
        x = self.conv4(x, edge_index)
        return F.log_softmax(x, dim=1)

In [86]:
def weights_init(m):
    if isinstance(m, GCNConv):
        torch.nn.init.xavier_uniform_(m.lin.weight.data)
        if m.lin.bias is not None:
            m.lin.bias.data.fill_(0.01)

In [87]:
y_train_label = torch.tensor(y_train.values, dtype=torch.int64)
y_train_label = torch.unsqueeze(y_train_label, dim=1)
y_test_label = torch.tensor(y_test.values, dtype=torch.int64)
y_test_label = torch.unsqueeze(y_test_label, dim=1)
y_train_label = y_train_label.view(-1)

In [88]:
data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
loader = DataLoader([data], batch_size=1)

In [186]:
# Initialize model and optimizer
gcn_model = GCN()
gcn_model.apply(weights_init)
optimizer = torch.optim.Adam(gcn_model.parameters(), lr=0.01)

In [89]:
class_weights = 1.0 / torch.tensor(label_prob)
class_weights = class_weights / class_weights.sum()  # Normalize

class_weights = class_weights.to(torch.float).to('cuda' if torch.cuda.is_available() else 'cpu')

In [90]:
gcn_model = torch.load('/content/drive/MyDrive/RAC Project/model/risk_lvl_5.pth')

In [188]:
# Training
gcn_model.train()
for epoch in range(100000):
    optimizer.zero_grad()
    for data in loader:
        out = gcn_model(data)

        loss = F.cross_entropy(out, y_train_label, weight=class_weights)

        # Backpropagation
        loss.backward()
        optimizer.step()

    if (epoch+1)%1000 == 0:
        torch.save(gcn_model, '/content/drive/MyDrive/RAC Project/model/risk_lvl_5.pth')
        print("Epoch "+str(epoch+1)+", loss: "+str(loss))

In [192]:
# G_test = nx.Graph(crs='EPSG:4326')
# # G_test.crs
# for idx, row in test_nodes.iterrows():
#     G_test.add_node(idx, x=row['x'], y=row['y'])
# for idx, row in test_edges.iterrows():
#     G_test.add_edge(row.name[0], row.name[1], oneway=row['oneway'])

# G_test.add_edges_from(nx.selfloop_edges(G_test))

# x_test = torch.tensor(test_nodes[['x', 'y', 'street_count']].values, dtype=torch.float)
# test_edges['oneway'] = test_edges['oneway'].astype(float)
# edge_index_test = torch.tensor(np.array(list(G_test.edges())).T, dtype=torch.long)
# edge_index_test = edge_index_test.remainder(len(test_nodes))  # Ensure node indices are within bounds
# edge_attr_test = torch.tensor(test_edges[['oneway', 'length', 'speed_kph']].values, dtype=torch.float)

# # Create DataLoader for test data
# test_data = Data(x=x_test, edge_index=edge_index_test, edge_attr=edge_attr_test)
# test_loader = DataLoader([test_data], batch_size=1)

Predict the risk level of all the nodes in our study area.

In [91]:
x_perth = torch.tensor(nodes[['x', 'y', 'street_count']].values, dtype=torch.float)
perth_edges = edges.copy()
perth_edges['oneway'] = perth_edges['oneway'].astype(float)
edge_index_perth = torch.tensor(np.array(list(G.edges())).T, dtype=torch.long)
edge_index_perth = edge_index_perth.remainder(len(nodes))  # Ensure node indices are within bounds
edge_attr_perth = torch.tensor(perth_edges[['oneway', 'length', 'speed_kph']].values, dtype=torch.float)

# Create DataLoader for the entire perth
perth_data = Data(x=x_perth, edge_index=edge_index_perth, edge_attr=edge_attr_perth)
perth_loader = DataLoader([perth_data], batch_size=1)

In [92]:
gcn_model.eval()

# Perform predictions
predictions = []
for data in perth_loader:
    with torch.no_grad():
        out = gcn_model(data)
        predicted_labels = torch.argmax(out, dim=1)
        # Append the predicted labels for the entire batch to the predictions list
        predictions.extend(predicted_labels.cpu().numpy())

Add the risk level into the street network map and save (graph and csv).

In [93]:
predictions_series = pd.Series(predictions, index=nodes.index)
nodes['risk_lvl'] = predictions_series

In [94]:
# Save to a graphml
pred_G = ox.graph_from_gdfs(nodes, edges)
ox.save_graphml(pred_G, '/content/drive/MyDrive/RAC Project/map/perth-drive-con-unprojected-risk.graphml')

In [95]:
selected_columns = nodes[['osmid_original', 'risk_lvl']]
selected_columns = selected_columns.rename(columns={'osmid_original': 'osmid'})

# Save to a CSV file
output_csv_path = '/content/drive/MyDrive/RAC Project/map/risk_lvl.csv'
selected_columns.to_csv(output_csv_path, index=False)